# Descripción.
El cáncer de mama es uno de los más comunes entre las mujeres. Los métodos de clasificación y minería de datos (data mining) que se presentan resultan ser de gran utilidad para el diagnóstico y la toma de decisiones médicas.

# Conjunto de datos.
Vamos a realizar un análisis de la base de datos Wisconsin de diagnóstico sobre cáncer de mama creada por el Dr. William H. Wolberg del hospital universitario de Wisconsin (Estados Unidos). 
Para crear la base de datos, Dr. Wolberg utilizó muestras obtenidas por punción aspiración con aguja fina de nódulo sólidos de mama. Se utilizó un programa de análisis gráfico de imágenes llamado Xcyt, que obtuvo las variables mediante un análisis citológico basado en un escaneo digital de las muestras de los nódulos. El programa utiliza un algoritmo de ajuste de curva para calcular 10 variables de cada una de las células de la muestra. Finalmente, el programa, calcula el valor medio (mean), el valor extremo (worst) y el error estándar (se) de cada muestra de la imagen devolviendo así 30 variables.

<img src="_img/breast_img.png">

Datos disponibles en: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
# Variables.
1) id: código de la paciente

2) diagnosis: diagnóstico (M = maligno, B = benigno) 

3-32) Diez valores numéricos calculados para cada núcleo celular con valor medio (mean), valor extremo (worst) y error estándar (se):
- radius_: distancia del centro al perímetro
- texture_: desviación estándar de los valores de la escala de grises
- perimeter_: perímetro
- area_: área
- smoothness_: variación local de las longitudes de los radios
- compactness_: (perimeter² / area - 1.0)
- concavity_:  gravedad (severity) de las zonas cóncavas del contorno
- concave.points_ (number of concave portions of the contour)
- symmetry_: simetría
- fractal_dimension_: (“coastline approximation” - 1)

33) Una columna con valores NA

# Objetivo.
El presente análisis de datos pretende observar qué variables son las más útiles para predecir si un tumor es maligno o benigno.

In [ ]:
# Cargamos librerías
# Si alguna no está instalada en su equipo use este código
# if (!require("ggplot2")) install.packages("ggplot2")
library("ggplot2")
library("caTools")
library("corrplot")
library("dplyr")
library("reshape")
library("caret")
library("GGally")
library(funModeling)

# Estadística Descriptiva
Primer paso para hacernos una idea de los datos

In [ ]:
# Cargamos los datos
library("readr")
data <- read.csv("_data/data.csv", sep=",")

In [ ]:
# Primer vistazo a los datos
head(data)
glimpse(data)

Tenemos. datos de 569 pacientes con 33 variables para cada paciente descritas anteriormente. Recordar que las 10 variables numéricas se obtienen para la media (mean), el valor extremo (worst) y el error estándar (se).

In [ ]:
# Estructura de los datos
str(data)

In [ ]:
# Dimensión del conjunto de datos
dim(data)

In [ ]:
# Resumen del conjunto de datos
summary(data)

# Procesado de datos
Los datos en crudo los tenemos almacenados con el nombre `data`. Vamos a realizar un procesado de datos almacenando lo obtenido con el nombre `data2`.

In [ ]:
# La primera columna id no nos interesa por lo que la borramos y vemos
# el encabezado y los primeros registros y la dimensión (filas y columnas).

data2 <- data[,-c(0:1)]
head(data2)
dim(data2)

In [ ]:
# La última columna X tampoco nos es útil La eliminamos
data2 <- data2[,-32]
head(data2)
dim(data2)

In [ ]:
# Arreglamos los datos para tener variables numéricas expecto la variable
# diagnosis que nos interesa como factor
data2$diagnosis <- as.factor(data2$diagnosis)
str(data2)

In [ ]:
# Ahora buscamos valores no válidos (NA, ?, etc.)
sapply(data2, function(x) sum(is.na(x)))

Vemos que no hay datos NA en todas las variables 
De este modo, en `data2` tenemos alamacenados

- 569 registros, cada uno con 31 variables.
- Diagnosis es una variable categórica (B = benigno, M = maligno)
- Todos los valores numéricos tienen 4 decimales
- Ausencia de NA

Una de las razones principales de hacer un análisis exploratorio de los datos es visualizar qué variables son las más significativas para predecir el cáncer de mama de cara a crear un modelo.

In [ ]:
# Análisis de datos: Número de pacientes afectadas. 
# Estadio Benigno y Maligno
data2 %>% count(diagnosis)

# Visualización de datos

In [ ]:
# Visualización de datos. Porcentaje de diagnóstico de cáncer
diagnosis.table <- table(data2$diagnosis)
colors <- terrain.colors(2) 
# Creamos el gráfico de sectores
diagnosis.prop.table <- prop.table(diagnosis.table)*100
diagnosis.prop.df <- as.data.frame(diagnosis.prop.table)
pielabels <- sprintf("%s - %3.1f%s", diagnosis.prop.df[,1], diagnosis.prop.table, "%")
pie(diagnosis.prop.table,
  labels=pielabels,  
  clockwise=TRUE,
  col=colors,
  border="gainsboro",
  radius=0.8,
  cex=0.8, 
  main="Porcentaje de diagnóstico de cáncer")
legend(1, .4, legend=diagnosis.prop.df[,1], cex = 0.7, fill = colors)

Vamos a visualizar las variables en forma de histograma discriminando malignos y benignos.
Antes, separamos las columnas por grupos de acuerdo a los sufijos de las variables (_mean, _se y _worst) para visulizarlos por esos grupos.

In [ ]:
# Separamos en 3 grupos
data2_mean <- data2[ ,c("diagnosis", "radius_mean", "texture_mean","perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave.points_mean", "symmetry_mean", "fractal_dimension_mean" )]

data2_se <- data2[ ,c("diagnosis", "radius_se", "texture_se","perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se", "concave.points_se", "symmetry_se", "fractal_dimension_se" )]

data2_worst <- data2[ ,c("diagnosis", "radius_worst", "texture_worst","perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst", "concave.points_worst", "symmetry_worst", "fractal_dimension_worst" )]


In [ ]:
# Obtenemos los histogramas del grupo "_mean" separados por diagnóstico
ggplot(data = melt(data2_mean, id.var = "diagnosis"), mapping = aes(x = value)) + 
  geom_histogram(bins = 10, aes(fill=diagnosis), alpha=0.5) + facet_wrap(~variable, scales =      'free_x')


In [ ]:
# Obtenemos los histogramas del grupo "_se" separados por diagnóstico
ggplot(data = melt(data2_se, id.var = "diagnosis"), mapping = aes(x = value)) + 
  geom_histogram(bins = 10, aes(fill=diagnosis), alpha=0.5) + facet_wrap(~variable, scales = 'free_x')


In [ ]:
# Obtenemos los histogramas del grupo "_worst" separados por diagnóstico
ggplot(data = melt(data2_worst, id.var = "diagnosis"), mapping = aes(x = value)) + 
  geom_histogram(bins = 10, aes(fill=diagnosis), alpha=0.5) + facet_wrap(~variable, scales = 'free_x')


A la vista de los gráficos obtenidos:
- Vemos que la mayoría de las variables están normalmente distribuidas.
- La comparación de las distribuciones por diagnóstico no muestra una separación perfecta en ninguna de las variables, es decir, que por sí solas no discriminan entre benigno y maligno.
- Parece que la hay en las variables `concave.points_worst`, `concavity_worst`, `perimeter_worst`, `area_mean` y `perimeter_mean`.
- Parece haber una superposición total en `symmetry_se` `smoothness_se`.

Vamos a visualizar con más detalle los datos correspondientes a la variable `area_mean`

In [ ]:
# Diagrama de cajas y bigotes de la variable area_mean
# separado por diagnóstico (benigno, maligno)
ggplot(data=data2,aes(x=diagnosis,y=area_mean,fill="pink"))+geom_boxplot()+ggtitle("area_mean Benigno Vs Maligno")

In [ ]:
# Gráficos de densidad de la variable area_mean separados por diagnóstico
ggplot(data2,aes(x=area_mean,fill=as.factor(diagnosis)))+geom_density(alpha=0.4)+ggtitle(" area_mean por diagnóstico")

In [ ]:
# Diagramas de violín de la variable area_mean
ggplot(data2,aes(x=as.factor(diagnosis),y=area_mean))+geom_violin()+ggtitle(" area_mean para benigno y maligno")

In [ ]:
# Diagramas de barras para area_mean mayor que 500
ggplot(data2, aes(x = area_mean>500, fill = diagnosis)) +geom_bar(position = "fill")+ggtitle("area_mean TRUE > 500 ")

In [ ]:
# Visualizamos la variable area_mean por valores. Recordamos que es la columna 5
library(funModeling)
cross_plot(data2[,c(1,5)], input=names(data2[,c(1,5)]), target="diagnosis")

A la vista de estos gráficos y de nuestros datos, vemos que la probabilidad de ser diagnosticado con un tumor maligno aumenta a medida que aumenta el área media (`area_mean`). Este resultado muestra que cuando el resultado del paciente está en el intervalo (921-1192), la probabilidad de ser diagnosticado con un tumor maligno es del 98,2%. 

**Ejercicio**: pruebe a cambiar valores en el código anterior (número de la columna) y visualizar los gráficos de otras variables como `concave.points_worst`(columna 29) o `symmetry_se`(columna 20).